In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
from rich import print
import warnings
import datetime
from astropy.time import Time
import os

In [2]:
header_data = pd.read_csv('../ref/mislike_image_header_table.csv')
catalog_filename = '../ref/catalog_nd_daostarfinder.parquet'
catalog_file = pq.ParquetFile(catalog_filename)

In [30]:
#os.listdir('/Users/cm/GFCAT/gfcat/test_known_jpg/sorted/moving/' <= these are fast satellites, ignore
tbl = pd.DataFrame({'eclipse':[],'datetime_iso':[], 'datetime_decimal':[],
                    'ra':[],'dec':[]})
for fn in os.listdir('/Users/cm/GFCAT/gfcat/test_gifs/sorted/moving/'):
    eclipse = int(fn.split('-')[0][1:])
    obj_id = int(fn.split('-')[1])
    aspect = pq.read_table('/Users/cm/github/gphoton_working/gPhoton/aspect/aspect.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()
    obstart = aspect.loc[aspect['flags']%2==0]['time'].min()
    #obsend = aspect.loc[aspect['flags']%2==0]['time'].max()
    #obsdur = obsend-obstart
    this_star = pq.read_table(catalog_filename,filters =
                          [('eclipse','=',eclipse),
                           ('obj_id','=',obj_id)]).to_pandas()

    GPSSECS = 315532800 + 432000
    t = obstart + GPSSECS
    dt = datetime.datetime.fromtimestamp(t)
    tbl = pd.concat([tbl,pd.DataFrame(
        {'eclipse':[eclipse],'datetime_iso':[str(dt)[:-7]],
         'datetime_decimal':[Time(dt,format='datetime').decimalyear],
         'ra':[this_star['ra'].values[0]],'dec':[this_star['dec'].values[0]]})])

tbl = tbl.sort_values('eclipse')
tbl[['eclipse','datetime_iso','datetime_decimal','ra','dec']].to_csv('gfcat_moving_objects.csv',index=None)